In [1]:
%load_ext jupyter_black

In [2]:
import json
from glob import glob
from pathlib import Path
from typing import NewType, Iterable, Callable

import nvector
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import nvector as nv

# ml
import tensorflow
from sklearn.manifold import LocallyLinearEmbedding

# plotting
import matplotlib.pyplot as plt

# number of seconds in 2 mins
TWO_MINS = 120.0
idx: slice = pd.IndexSlice

FeatureCollection = NewType("FeatureCollection", dict)

wgs84 = nv.FrameE(name="WGS84")

all_files = sorted(glob("/workspaces/sppp/data/probsevere/*.json"))

In [35]:
# data transformation and initial loading
def open_file(filepath: Path) -> FeatureCollection:
    with filepath.open("rb") as fin:
        return json.load(fin)


def to_dataframe(fc: FeatureCollection) -> pd.DataFrame:
    df = gpd.GeoDataFrame.from_features(fc["features"])
    df["validTime"] = pd.to_datetime(fc["validTime"], format="%Y%m%d_%H%M%S %Z")
    df["CENTROID"] = df["geometry"].centroid

    def ecef_vector():
        for point in df["geometry"].centroid:
            geo_point = wgs84.GeoPoint(
                longitude=point.x, latitude=point.y, degrees=True
            )
            yield geo_point.to_ecef_vector()

    df["ECEF_VECTOR"] = tuple(ecef_vector())

    df = df.set_index(["validTime", "ID"])
    return df
    # df =


def to_midf() -> pd.DataFrame:
    def generate():
        for file in all_files:
            fc = open_file(Path(file))
            yield to_dataframe(fc)

    return pd.concat(generate())


midf = to_midf()
midf

geometry  \
validTime                 ID                                                         
2021-10-11 00:00:53+00:00 89234  POLYGON ((-96.06000 38.40000, -95.99000 38.400...   
                          89321  POLYGON ((-80.66000 24.76000, -80.62000 24.760...   
                          89467  POLYGON ((-96.98000 36.80000, -96.87000 36.800...   
                          89470  POLYGON ((-75.15000 35.44000, -75.11000 35.440...   
                          89519  POLYGON ((-98.29000 35.59000, -98.27000 35.590...   
...                                                                            ...   
2021-10-11 01:00:58+00:00 90389  POLYGON ((-97.10000 37.72000, -97.05000 37.720...   
                          90390  POLYGON ((-98.03000 34.15000, -97.99000 34.100...   
                          90391  POLYGON ((-98.87000 31.84000, -98.85000 31.830...   
                          90392  POLYGON ((-99.03000 31.68000, -99.01000 31.680...   
                          90393  POLYGON ((-99.35000 31.38000, -99.31000 31.380...   

                                MUCAPE MLCAPE MLCIN EBSHEAR SRH01KM  \
validTime                 ID                                          
2021-10-11 00:00:53+00:00 89234    485      0     0    48.6      63   
                          89321   2691   2179    -2    22.9      14   
                          89467   1367    937   -19    61.9     148   
                          89470   2493   2076     0    24.0      19   
                          89519   2035   1656   -17    74.6     197   
...                                ...    ...   ...     ...     ...   
2021-10-11 01:00:58+00:00 90389    576      0     0    52.1      74   
                          90390   2113   2067   -31    59.3     192   
                          90391   2362   1910   -22    67.5     197   
                          90392   2208   1744   -15    66.8     170   
                          90393   1812   1428   -32    58.1      88   

                                MEANWIND_1-3kmAGL  MESH VIL_DENSITY  \
validTime                 ID                                          
2021-10-11 00:00:53+00:00 89234              12.9  0.25        1.67   
                          89321               4.0  0.43        1.60   
                          89467              17.6  1.43        2.85   
                          89470              16.3  0.05        0.62   
                          89519              26.4  2.19        3.29   
...                                           ...   ...         ...   
2021-10-11 01:00:58+00:00 90389               8.4  0.00        0.52   
                          90390              36.4  0.25        0.88   
                          90391              32.8  0.04        0.77   
                          90392              32.2  0.59        1.75   
                          90393              31.5  0.48        1.68   

                                FLASH_RATE  ... PWAT CAPE_M10M30  LJA  SIZE  \
validTime                 ID                ...                               
2021-10-11 00:00:53+00:00 89234         10  ...  1.5         129  0.0   515   
                          89321          7  ...  1.4         668  0.0    45   
                          89467         60  ...  1.7         319  1.2   355   
                          89470          0  ...  2.3         420  0.0    50   
                          89519        181  ...  1.9         603  1.2  1685   
...                                    ...  ...  ...         ...  ...   ...   
2021-10-11 01:00:58+00:00 90389          0  ...  1.5         160  0.0   124   
                          90390          4  ...  1.9         704  0.0    48   
                          90391          0  ...  1.6         635  0.0    91   
                          90392         12  ...  1.5         594  0.0   210   
                          90393         15  ...  1.2         484  0.0   415   

                                        AVG_BEAM_HGT MOTION_EAST MOTION_SOUTH  

In [80]:
# helper function
from sklearn.manifold import LocallyLinearEmbedding
from sppp.transfer.const import GridSpace


grid = GridSpace()


def embed(df: pd.DataFrame, grid: GridSpace) -> pd.DataFrame:

    lle = LocallyLinearEmbedding(n_components=1, n_neighbors=10)
    fresh = df[["PS", "MOTION_EAST", "MOTION_SOUTH"]].copy()
    cent = df["geometry"].centroid
    fresh["STAB"] = lle.fit_transform(midf[["MUCAPE", "MLCAPE", "MLCIN"]])
    fresh = fresh.astype(np.float32)

    fresh["X"] = np.argmin(abs(cent.x.values[:, np.newaxis] - grid.x), axis=1)
    fresh["Y"] = np.argmin(abs(cent.y.values[:, np.newaxis] - grid.y), axis=1)

    return fresh


fresh = midf.pipe(embed, grid)

fresh

PS  MOTION_EAST  MOTION_SOUTH      STAB  \
validTime                 ID                                                  
2021-10-11 00:00:53+00:00 89234   11.0       15.042       -14.019  0.001099   
                          89321    4.0        0.423        -2.061  0.000797   
                          89467   98.0        9.610       -11.265 -0.002434   
                          89470    2.0       -1.606         0.677  0.000946   
                          89519  100.0       12.789        -8.273 -0.000547   
...                                ...          ...           ...       ...   
2021-10-11 01:00:58+00:00 90389    4.0       11.984       -13.031  0.001101   
                          90390   33.0       13.982        -3.008  0.000532   
                          90391   10.0       17.650        -6.134  0.001023   
                          90392   64.0       13.725        -0.784 -0.000151   
                          90393   47.0       12.362        -0.220 -0.003112   

                                    X     Y  
validTime                 ID                 
2021-10-11 00:00:53+00:00 89234  3393  1676  
                          89321  4936  3027  
                          89467  3305  1828  
                          89470  5487  1960  
                          89519  3176  1987  
...                               ...   ...  
2021-10-11 01:00:58+00:00 90389  3290  1734  
                          90390  3197  2090  
                          90391  3110  2321  
                          90392  3088  2346  
                          90393  3047  2384  

[2232 rows x 6 columns]

In [5]:
# machine learning support classes
from tensorflow import keras
from keras.engine.sequential import Sequential


class State:
    __has_state: bool = False

    def __init__(self):
        self.__latests: pd.DataFrame = None
        self.__state: pd.DataFrame = None

    def __repr__(self) -> str:
        return self.__state.__repr__()

    @property
    def latests(self) -> pd.DataFrame:
        return self.__latests

    def set_storm(self, df: pd.DataFrame) -> None:

        self.__latests = df

        if not self.__has_state:
            self.__has_state = True
            self.__state = df
        else:
            old = self.__state
            self.__state = pd.concat([old, df])

    def iterstorms(self):
        df = self.__state.iloc[-2:].groupby("ID")
        for id, x in self.__state.iloc[-2:].groupby("ID"):
            yield x

    def has_hist(self) -> bool:
        return isinstance(self.__state, pd.DataFrame)

    @property
    def frame(self) -> pd.DataFrame:
        return self.__state


def build_model(frame_b: pd.DataFrame) -> Sequential:
    model = keras.models.Sequential(
        [
            keras.layers.Dense(
                22.5,
                activation="elu",
                input_shape=frame_b.shape,
            ),
            keras.layers.Dense(
                22.5,
                activation="sigmoid",
            ),
            keras.layers.Dense(3),
        ]
    )
    return model

In [6]:
import tensorflow as tf
import gym

from sppp.transfer.funcs import mask_frames_by_id
from sppp.transfer.const import GridSpace

grid = GridSpace()


class SPPPEnv(gym.Env):
    def __init__(self, state: "State") -> None:
        self.state = state

    def _compute_reward(self) -> int:
        return 1

    def step(self, action: np.ndarray):
        # action is produced by DQN, action is discrete
        # self.cache.move(action)
        # compute reward based on state(position) of the car
        # storm_state = self.car_agent.getCarState()
        # reward = self._compute_reward(storm_state)
        # # check if the episode is done
        # car_controls = self.car_agent.getCarControls()
        # done = self._isDone(storm_state, car_controls, reward)
        # # log info
        # info = {}
        # # observation is RGB image from car's camera
        # observation = self.car_agent.observe()
        observation = self.state.latests
        reward = self._compute_reward()
        done = False
        info = {}
        return observation, reward, done, info

    @property
    def observation_space(self) -> pd.DataFrame:
        return self.state.latests


def iterframe(df: pd.DataFrame) -> Iterable[tuple[pd.Timestamp, pd.DataFrame]]:
    yield from df.groupby("validTime")


n_inputs = 4
state = State()
env = SPPPEnv(state)
loss = keras.losses.binary_crossentropy


def play_one_step(
    env: SPPPEnv,
    obs: np.ndarray,
    model: Sequential,
    loss_fn: Callable[[any], any],
):
    """policy gradient"""
    with tf.GradientTape() as tape:
        left_prob = model(obs[np.newaxis])
        action = tf.random.uniform([1, 1]) > left_prob

        y_target = tf.constant([[1.0]]) - tf.cast(action, tf.float32)

        loss = tf.reduce_mean(loss_fn(y_target, left_prob))

    grads = tape.gradient(loss, model.trainable_variables)
    return action.numpy() 


n_outputs = 5

if __name__ == "__main__":
    for vt, df in iterframe(fresh):
        track_rewards = 0
        obs = env.reset()
        # evaluate prediction
        if state.has_hist():
            # there is existing storm information normalize frame_a and frame_b by the the in's in the index
            frame_a, frame_b = mask_frames_by_id(state.latests, df)
            # assert that the frames are of an equal shape
            assert frame_a.shape == frame_b.shape
            bg = frame_b[["MOTION_EAST", "MOTION_SOUTH", "STAB"]].values

            model = build_model(bg)

            x = play_one_step(env, bg, model, loss)
            # obs, reward, done, info = env.step(frame_b[["MOTION_EAST", "MOTION_SOUTH"]])
        # make a prediction
        state.set_storm(df)

# frame_b[["MOTION_EAST", "MOTION_SOUTH"]] * TWO_MINSb

2022-07-20 16:14:38.171769: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-20 16:14:38.171891: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2c93fd1564f7): /proc/driver/nvidia/version does not exist
2022-07-20 16:14:38.172516: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
df = state.frame.copy()
df
# df["ECEF_VECTOR"] + (df[["X", "Y"]] * TWO_MINS).stack().values[:, np.newaxis]

PS  MOTION_EAST  MOTION_SOUTH  \
validTime                 ID                                        
2021-10-11 00:00:53+00:00 89234   11.0       15.042       -14.019   
                          89321    4.0        0.423        -2.061   
                          89467   98.0        9.610       -11.265   
                          89470    2.0       -1.606         0.677   
                          89519  100.0       12.789        -8.273   
...                                ...          ...           ...   
2021-10-11 01:00:58+00:00 90389    4.0       11.984       -13.031   
                          90390   33.0       13.982        -3.008   
                          90391   10.0       17.650        -6.134   
                          90392   64.0       13.725        -0.784   
                          90393   47.0       12.362        -0.220   

                                                                       ECEF_VECTOR  \
validTime                 ID                                                         
2021-10-11 00:00:53+00:00 89234  ECEFvector(pvector=[[-530266.6328842767], [-49...   
                          89321  ECEFvector(pvector=[[943548.7176692891], [-571...   
                          89467  ECEFvector(pvector=[[-619075.6783992505], [-50...   
                          89470  ECEFvector(pvector=[[1335819.2918418334], [-50...   
                          89519  ECEFvector(pvector=[[-748148.9557063201], [-51...   
...                                                                            ...   
2021-10-11 01:00:58+00:00 90389  ECEFvector(pvector=[[-624810.164130543], [-501...   
                          90390  ECEFvector(pvector=[[-737892.7786927632], [-52...   
                          90391  ECEFvector(pvector=[[-839030.8095662066], [-53...   
                          90392  ECEFvector(pvector=[[-862084.2989790718], [-53...   
                          90393  ECEFvector(pvector=[[-904281.970253052], [-538...   

                                     STAB          X          Y  
validTime                 ID                                     
2021-10-11 00:00:53+00:00 89234  0.000910 -96.068283  38.238827  
                          89321 -0.000710 -80.632050  24.728102  
                          89467 -0.002573 -96.946304  36.715923  
                          89470 -0.000142 -75.129051  35.398010  
                          89519 -0.001236 -98.236542  35.127361  
...                                   ...        ...        ...  
2021-10-11 01:00:58+00:00 90389  0.000893 -97.099228  37.659824  
                          90390 -0.000599 -98.022614  34.098648  
                          90391  0.000240 -98.894226  31.783758  
                          90392 -0.000619 -99.115829  31.531752  
                          90393 -0.003379 -99.528023  31.157946  

[2232 rows x 7 columns]